In [1]:
import pandas as pd

In [2]:
#Returns train/test dataframe
def Aggregate_C1(dataset):    
    
    # Create new action columns to the dataset and add 1 for each action that takes place.
    # Add device_count column
    
    ds = pd.read_csv(f"/home/jovyan/shared/datasets/RetailBanking/{dataset}")
    accounts_all = pd.read_csv("/home/jovyan/shared/datasets/RetailBanking/accounts_all.csv")
    customers_all = pd.read_csv("/home/jovyan/shared/datasets/RetailBanking/customers_all.csv")
    devices_all = pd.read_csv("/home/jovyan/shared/datasets/RetailBanking/devices_all.csv")
    device_sessions_all = pd.read_json("/home/jovyan/shared/datasets/RetailBanking/device_sessions_all.json")

    devices_summary = (
    devices_all.groupby("CustomerID")["DeviceID"]
    .agg(
        Device_List=lambda x: ",".join(set(x.astype(str))),  # unique IDs as string
        Device_Count="nunique"  # count of unique devices
    )
    .reset_index()
)

    merged = pd.DataFrame()
    
    #Fraud Detection
    
    # When testing this func, check if device sessions is merged properly 
    merged = (
        ds
        .merge(accounts_all, left_on="SrcAccount", right_on="AccountID", how="left", suffixes=("", "_src"))
        .merge(accounts_all, left_on="DstAccount", right_on="AccountID", how="left", suffixes=("", "_dst"))
        .merge(customers_all, on="CustomerID", how="left")
        .merge(devices_summary, on="CustomerID", how="left")
        .merge(device_sessions_all, on="SessionID", how="left")
    )

    merged["SessionID"] = merged["SessionID"].notna().astype(int)

    count_actions = lambda actions, name: sum(
    1 for a in (actions if isinstance(actions, list) else []) 
    if a == name or (isinstance(a, dict) and a.get('type') == name)
)
    
    merged["Action_Count"] = merged["Actions"].apply(lambda x: len(x) if isinstance(x, list) else 0)
    merged["Login_Count"] = merged["Actions"].apply(lambda x: count_actions(x,"login"))
    merged["Logout_Count"] = merged["Actions"].apply(lambda x: count_actions(x,"logout"))
    merged["Account_View_Count"] = merged["Actions"].apply(lambda x: count_actions(x,"account_view"))
    merged["Payment_Count"] = merged["Actions"].apply(lambda x: count_actions(x,"payment"))
    merged["Transfer_Count"] = merged["Actions"].apply(lambda x: count_actions(x,"transfer"))
    
    td = pd.to_datetime(merged["Timestamp_dt"], format="%Y/%m/%d %H:%M")
    merged["Day_of_Week"] = td.dt.day_name()
    merged["Timestamp"] = td.dt.time
    merged["Timestamp_dt"] = td.dt.date    

    merged.rename(columns={"CustomerID_x": "CustomerID", "SessionID":"Online"}, inplace=True)
    #merged_dropped = merged.drop(columns=["TxnID", "SrcAccount", "DstAccount","CustomerID_src",'CustomerID_y', "CustomerID_dst", 'Timestamp_y',"DeviceID_y","Timestamp_x"])
    
    return merged

In [6]:
#Aggregate_C1("transactions_train.csv")

,Timestamp_dt,TxnID,Timestamp_x,Online,CustomerID,SrcAccount,DstAccount,Channel,MCC_Group,Amount,...,DeviceID,Actions,Action_Count,Login_Count,Logout_Count,Account_View_Count,Payment_Count,Transfer_Count,Day_of_Week,Timestamp
0,2025-01-01,532ceea0-846f-4689-80e7-953913c8cbd5,2025/1/1 0:00,0,C000352,A92e83abd,NaN,ATM,Groceries,37.837264,...,NaN,NaN,0,0,0,0,0,0,Wednesday,00:00:00
1,2025-01-01,cdaeae81-0137-426a-92c7-c88726184fa7,2025/1/1 0:02,0,C000965,A3f209383,NaN,ATM,Salary/Benefit,71.308076,...,NaN,NaN,0,0,0,0,0,0,Wednesday,00:02:00
2,2025-01-01,68cfb36d-e2a9-4f70-ad4b-3f0029cd17cb,2025/1/1 0:03,0,C000441,Adb76359d,NaN,ATM,Other,53.406819,...,NaN,NaN,0,0,0,0,0,0,Wednesday,00:03:00
3,2025-01-01,a5c24ed3-55d7-487a-8de9-cd2e95235a1b,2025/1/1 0:04,0,C000582,A4ae7d26c,NaN,ATM,Groceries,26.447249,...,NaN,NaN,0,0,0,0,0,0,Wednesday,00:04:00
4,2025-01-01,e4d77002-c783-4785-9bdc-e6bf19c3144c,2025/1/1 0:05,0,C000973,A5b5e1a2c,NaN,ATM,Other,87.814168,...,NaN,NaN,0,0,0,0,0,0,Wednesday,00:05:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152310,2025-06-30,74c43782-8db9-45a1-8d24-6fee3dcb5339,2025-06-30T23:38:55,1,C000396,A187398b2,A847214c8,online,Utilities,-26.180904,...,D5509F,"[login, {'type': 'account_view'}, {'type': 'pa...",5,1,1,1,1,1,Monday,23:38:00
152311,2025-06-30,b9a50adb-d52b-4035-a805-78a2dd42042f,2025-06-30T23:38:55,1,C000396,A187398b2,NaN,online,Other,-82.901811,...,D5509F,"[login, {'type': 'account_view'}, {'type': 'pa...",5,1,1,1,1,1,Monday,23:38:00
152312,2025-06-30,cd97a824-65a5-4dfd-947d-923cc54a22de,2025/6/30 23:51,0,C000846,A56d308e5,NaN,ATM,Online Retail,33.790693,...,NaN,NaN,0,0,0,0,0,0,Monday,23:51:00
152313,2025-06-30,9ae1b191-e4d1-4de0-841e-245ee6b09c33,2025/6/30 23:55,0,C000703,Aa25d58cc,NaN,branch,Groceries,51.861944,...,NaN,NaN,0,0,0,0,0,0,Monday,23:55:00


In [8]:
agg_df = Aggregate_C1("transactions_test.csv")
agg_df.to_csv('aggtest.csv')

In [186]:
agg_df.to_csv('agg3.csv')

In [5]:
# Load transaction data
# Did not import customer panel train and test set since its probably meant for C1 
train_txns = pd.read_csv("/home/jovyan/shared/datasets/RetailBanking/transactions_train.csv")
test_txns = pd.read_csv("/home/jovyan/shared/datasets/RetailBanking/transactions_test.csv")
accounts_all = pd.read_csv("/home/jovyan/shared/datasets/RetailBanking/accounts_all.csv")
customers_all = pd.read_csv("/home/jovyan/shared/datasets/RetailBanking/customers_all.csv")
devices_all = pd.read_csv("/home/jovyan/shared/datasets/RetailBanking/devices_all.csv")
device_sessions_all = pd.read_json("/home/jovyan/shared/datasets/RetailBanking/device_sessions_all.json")

In [32]:
print(f"✅ Data loaded:")
print(f"   Train transactions: {len(train_txns)}")
print(f"   Test transactions: {len(test_txns)}")
print(f"   Customer columns: {list(train_txns.columns)}")

print(f"   Train columns: {list(train_txns.columns)}")
so = list(train_txns.columns)
print(f"   Test columns: {list(test_txns.columns)}")
print(f"   accounts_all columns: {list(accounts_all.columns)}")
print(f"   customers_all columns: {list(customers_all.columns)}")
print(f"   devices_all columns: {list(devices_all.columns)}")
print(f"   device_sessions_all columns: {list(device_sessions_all.columns)}")

common_cols_all = \
      set(accounts_all.columns) \
    & set(customers_all.columns) \
    & set(devices_all.columns) \
    & set(device_sessions_all.columns)

print("Common columns across all DataFrames:")
print(sorted(list(common_cols_all)))


✅ Data loaded:
   Train transactions: 152315
   Test transactions: 151587
   Train columns: ['Timestamp_dt', 'TxnID', 'Timestamp', 'SessionID', 'CustomerID', 'SrcAccount', 'DstAccount', 'Channel', 'MCC_Group', 'Amount', 'FraudLabel']
   Test columns: ['Timestamp_dt', 'TxnID', 'Timestamp', 'SessionID', 'CustomerID', 'SrcAccount', 'DstAccount', 'Channel', 'MCC_Group', 'Amount']
   accounts_all columns: ['CustomerID', 'AccountID', 'Type', 'Balance', 'Limit', 'OpenDate']
   customers_all columns: ['CustomerID', 'Age', 'Tenure', 'CreditScore', 'HomeCity', 'AnnualSalary']
   devices_all columns: ['CustomerID', 'DeviceID']
   device_sessions_all columns: ['CustomerID', 'SessionID', 'Timestamp', 'City', 'IP', 'DeviceID', 'Actions']
Common columns across all DataFrames:
['CustomerID']


In [8]:
#We need customer_all, accounts_all
accounts_all = pd.read_csv("/home/jovyan/shared/datasets/RetailBanking/accounts_all.csv")
customers_all = pd.read_csv("/home/jovyan/shared/datasets/RetailBanking/customers_all.csv")
customer_train = pd.read_csv("/home/jovyan/shared/datasets/RetailBanking/customer_panel_train.csv")
customer_test = pd.read_csv("/home/jovyan/shared/datasets/RetailBanking/customer_panel_test.csv")
accounts_all_grouped = accounts_all.groupby("CustomerID")["AccountID"].nunique()

customer_train_agg = (
    customer_train
    .merge(customers_all, on="CustomerID", how="left")
)
    
customer_test_agg = (
    customer_test
    .merge(customers_all, on="CustomerID", how="left")
)

In [20]:
#Don't Average it and keep it separate for each account somehow 
#Redo it
accounts_all_grped = (
    accounts_all
    .groupby("CustomerID")
    .agg(
        Avg_Balance=("Balance", "mean"),
        Avg_Credit_Limit=("Limit", "mean"),
        Account_Types=("Type", lambda x: ",".join(sorted(set(x.astype(str))))),
        Account_Count=("AccountID", "nunique"),
        First_Open_Date=("OpenDate", "min"),
        Last_Open_Date=("OpenDate", "max"),
    )
    .reset_index()
)
accounts_all_grped

,CustomerID,Avg_Balance,Avg_Credit_Limit,Account_Types,Account_Count,First_Open_Date,Last_Open_Date
0,C000001,535.150521,24081.773437,credit_card,1,2019-07-02,2019-07-02
1,C000002,169.444937,8485.566470,"checking,credit_card",2,2020-09-17,2020-11-22
2,C000003,1765.491528,194546.095136,"checking,credit_card",3,2024-06-23,2024-09-17
3,C000004,1175.699220,NaN,savings,1,2021-12-11,2021-12-11
4,C000005,300.342311,NaN,checking,1,2024-11-08,2024-11-08
...,...,...,...,...,...,...,...
995,C000996,518.215969,23319.718622,credit_card,2,2022-08-05,2022-10-19
996,C000997,1026.324145,NaN,savings,1,2021-04-13,2021-04-13
997,C000998,255.363386,11491.352383,credit_card,1,2020-07-24,2020-07-24
998,C000999,849.765832,31979.358719,"checking,credit_card,savings",3,2019-05-21,2022-12-17


In [11]:
type_balances = (
    accounts_all
    .pivot_table(
        index="CustomerID",
        columns="Type",
        values="Balance",
        aggfunc="mean"
    )
    .add_prefix("Avg_Balance_")
    .reset_index()
)
type_balances

Type,CustomerID,Avg_Balance_checking,Avg_Balance_credit_card,Avg_Balance_savings
0,C000001,NaN,535.150521,NaN
1,C000002,150.321730,188.568144,NaN
2,C000003,486.614013,4323.246559,NaN
3,C000004,NaN,NaN,1175.699220
4,C000005,300.342311,NaN,NaN
...,...,...,...,...
995,C000996,NaN,518.215969,NaN
996,C000997,NaN,NaN,1026.324145
997,C000998,NaN,255.363386,NaN
998,C000999,262.577659,710.652416,1576.067421


In [12]:
accounts_all_final = accounts_all_grped.merge(type_balances, on="CustomerID", how="left")

In [13]:
customer_train.merge(accounts_all_final,on="CustomerID", how="left")

,CustomerID,Avg_Balance,Avg_Credit_Limit,Account_Types,Account_Count,First_Open_Date,Last_Open_Date,Avg_Balance_checking,Avg_Balance_credit_card,Avg_Balance_savings
0,C000001,535.150521,24081.773437,credit_card,1,2019-07-02,2019-07-02,NaN,535.150521,NaN
1,C000002,169.444937,8485.566470,"checking,credit_card",2,2020-09-17,2020-11-22,150.321730,188.568144,NaN
2,C000003,1765.491528,194546.095136,"checking,credit_card",3,2024-06-23,2024-09-17,486.614013,4323.246559,NaN
3,C000004,1175.699220,NaN,savings,1,2021-12-11,2021-12-11,NaN,NaN,1175.699220
4,C000005,300.342311,NaN,checking,1,2024-11-08,2024-11-08,300.342311,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
995,C000996,518.215969,23319.718622,credit_card,2,2022-08-05,2022-10-19,NaN,518.215969,NaN
996,C000997,1026.324145,NaN,savings,1,2021-04-13,2021-04-13,NaN,NaN,1026.324145
997,C000998,255.363386,11491.352383,credit_card,1,2020-07-24,2020-07-24,NaN,255.363386,NaN
998,C000999,849.765832,31979.358719,"checking,credit_card,savings",3,2019-05-21,2022-12-17,262.577659,710.652416,1576.067421


In [19]:
customer_train_final = (
        customer_train
        .merge(accounts_all_final, on="CustomerID", how="left")
        .merge(customers_all, on="CustomerID", how="left")
    )
customer_train_final.head(20)

,CustomerID,Week,Utilisation,PaymentRatio,HardInquiries,DefaultLabel,Avg_Balance,Avg_Credit_Limit,Account_Types,Account_Count,First_Open_Date,Last_Open_Date,Avg_Balance_checking,Avg_Balance_credit_card,Avg_Balance_savings,Age,Tenure,CreditScore,HomeCity,AnnualSalary
0,C000005,1,0.0,0.0,1,0,300.342311,NaN,checking,1,2024-11-08,2024-11-08,300.342311,NaN,NaN,53.840444,7.325337,781.438783,City147,49665.187703
1,C000005,2,0.0,0.0,0,0,300.342311,NaN,checking,1,2024-11-08,2024-11-08,300.342311,NaN,NaN,53.840444,7.325337,781.438783,City147,49665.187703
2,C000005,3,0.0,0.0,1,0,300.342311,NaN,checking,1,2024-11-08,2024-11-08,300.342311,NaN,NaN,53.840444,7.325337,781.438783,City147,49665.187703
3,C000005,4,0.0,0.0,0,0,300.342311,NaN,checking,1,2024-11-08,2024-11-08,300.342311,NaN,NaN,53.840444,7.325337,781.438783,City147,49665.187703
4,C000005,5,0.0,0.0,0,0,300.342311,NaN,checking,1,2024-11-08,2024-11-08,300.342311,NaN,NaN,53.840444,7.325337,781.438783,City147,49665.187703
5,C000005,6,0.0,0.0,0,0,300.342311,NaN,checking,1,2024-11-08,2024-11-08,300.342311,NaN,NaN,53.840444,7.325337,781.438783,City147,49665.187703
6,C000005,7,0.0,0.0,0,0,300.342311,NaN,checking,1,2024-11-08,2024-11-08,300.342311,NaN,NaN,53.840444,7.325337,781.438783,City147,49665.187703
7,C000005,8,0.0,0.0,1,0,300.342311,NaN,checking,1,2024-11-08,2024-11-08,300.342311,NaN,NaN,53.840444,7.325337,781.438783,City147,49665.187703
8,C000005,9,0.0,0.0,0,0,300.342311,NaN,checking,1,2024-11-08,2024-11-08,300.342311,NaN,NaN,53.840444,7.325337,781.438783,City147,49665.187703
9,C000005,10,0.0,0.0,0,0,300.342311,NaN,checking,1,2024-11-08,2024-11-08,300.342311,NaN,NaN,53.840444,7.325337,781.438783,City147,49665.187703
